In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=b6d180dff72a14a6b81a81dafb45a93ecba22523f5849c14d84f8910e548c79d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from torch.cuda import is_available
from sentence_transformers import SentenceTransformer, util
import os
import time
import csv
import torch

if torch.cuda.is_available():
  print("GPU available and ready to go")

model_name = "quora-distilbert-multilingual"
model = SentenceTransformer(model_name)

url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
max_corpus_size = 100000

GPU available and ready to go


In [11]:
if not os.path.exists(dataset_path):
    print("Download dataset")
    util.http_get(url, dataset_path)

Download dataset


  0%|          | 0.00/58.2M [00:00<?, ?B/s]

In [7]:
import pandas as pd

df = pd.read_csv(r'./drive/MyDrive/quora_duplicate_questions.tsv',sep='\t')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [9]:
len(df)

404290

In [8]:
df['question1'].nunique()

290456

In [10]:
df['question2'].nunique()

299174

In [ ]:
# Alternate wat to extract data

#corpus_sentences = set()
#with open(dataset_path, encoding='utf8') as fIn:
    #reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    #for row in reader:
        #corpus_sentences.add(row['question1'])
        #if len(corpus_sentences) >= max_corpus_size:
            #break

        #corpus_sentences.add(row['question2'])
        #if len(corpus_sentences) >= max_corpus_size:
            #break

In [12]:
# Get all unique statements from the file
corpus_sentences = set()
for sentence1,sentence2 in zip(df['question1'].values,df['question2'].values):
  corpus_sentences.add(sentence1)
  corpus_sentences.add(sentence2)
  if len(corpus_sentences) >= max_corpus_size:
    break

corpus_sentences = list(corpus_sentences)
len(corpus_sentences)

100001

In [14]:
# Encode the corpus statements

embeddings = model.encode(corpus_sentences,
                          show_progress_bar=True,
                          convert_to_tensor=True)

print("Shape of embeddings:",embeddings.shape)

Batches:   0%|          | 0/3126 [00:00<?, ?it/s]

Shape of embeddings: torch.Size([100001, 768])


In [15]:
# Function that searches the corpus and prints results

def search(input_ques):
  start_time = time.time()
  ques_embedding = model.encode(input_ques,convert_to_tensor=True)
  hits = util.semantic_search(ques_embedding,embeddings)
  end_time = time.time()
  hits = hits[0] # get hits for the first query

  print("Input question:",input_ques)
  print("Results after {:.3f} seconds:".format(end_time-start_time))
  for hit in hits[:5]:
    print("\t{:.3f}\t{}".format(hit['score'],
                                corpus_sentences[hit['corpus_id']]))

In [16]:
# test the function

search("How can I learn python online")

Input question: How can I learn python online
Results after 0.028 seconds:
	0.974	What's the best way to learn python on my own?
	0.969	What are some of the best ways to learn python?
	0.967	What are some tips for learning python?
	0.965	What is the best online resources to learn Python?
	0.964	How do I start learning python web programming from scratch to advanced?


In [17]:
search("What are the best tourist spots in Australia")

Input question: What are the best tourist spots in Australia
Results after 0.018 seconds:
	0.979	What is the best place in Australia?
	0.969	What are some of the best places to vacation in Australia?
	0.967	What are the best places for vacation in Australia?
	0.944	What's the best place to party in Australia?
	0.940	What is your review of Visiting and Travel in Australia?


In [18]:
#German: How can I learn Python online?
search("Wie kann ich Python online lernen?")

Input question: Wie kann ich Python online lernen?
Results after 0.016 seconds:
	0.989	What is the best online resources to learn Python?
	0.980	Where I should learn Python?
	0.980	Where should I start at to learn about how to do Python?
	0.980	Which are the best resources to learn Python?
	0.979	What's the best way to learn Python?
